In [14]:
#Price Prediction Model - End-to-End ML Pipeline

# This notebook demonstrates how to build, train, evaluate, and save
# a Machine Learning model for predicting Flight Price using Scikit-learn.
# The code is adapted from a Python script version for easier exploration.

# ---

In [15]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

In [16]:
#Load Dataset
BASE_DIR = os.path.dirname(os.getcwd())  
DATA_PATH = os.path.abspath(
    os.path.join(BASE_DIR, "app", "data", "flightPricePrediction.csv")
)

print(DATA_PATH)

/Users/rohanbk/Documents/DataScienceProjects/predictive-pipeline/app/data/flightPricePrediction.csv


In [17]:
# Handle missing file errors gracefully
try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
    print(" Dataset loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f" The file {DATA_PATH} was not found. Ensure it exists in the 'data/' folder.")

# Display basic info
print("Columns:", df.columns.tolist())
df.head()

 Dataset loaded successfully.
Columns: ['Unnamed: 0', 'airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left', 'price']


Unnamed: 0   airline   flight source_city departure_time stops  \
0           0  SpiceJet  SG-8709       Delhi        Evening  zero   
1           1  SpiceJet  SG-8157       Delhi  Early_Morning  zero   
2           2   AirAsia   I5-764       Delhi  Early_Morning  zero   
3           3   Vistara   UK-995       Delhi        Morning  zero   
4           4   Vistara   UK-963       Delhi        Morning  zero   

    arrival_time destination_city    class  duration  days_left  price  
0          Night           Mumbai  Economy      2.17          1   5953  
1        Morning           Mumbai  Economy      2.33          1   5953  
2  Early_Morning           Mumbai  Economy      2.17          1   5956  
3      Afternoon           Mumbai  Economy      2.25          1   5955  
4        Morning           Mumbai  Economy      2.33          1   5955

In [18]:
print("Columns in dataset:")
print(df.columns.tolist())

# Peek at the first few rows
df.head()

Columns in dataset:
['Unnamed: 0', 'airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left', 'price']


Unnamed: 0   airline   flight source_city departure_time stops  \
0           0  SpiceJet  SG-8709       Delhi        Evening  zero   
1           1  SpiceJet  SG-8157       Delhi  Early_Morning  zero   
2           2   AirAsia   I5-764       Delhi  Early_Morning  zero   
3           3   Vistara   UK-995       Delhi        Morning  zero   
4           4   Vistara   UK-963       Delhi        Morning  zero   

    arrival_time destination_city    class  duration  days_left  price  
0          Night           Mumbai  Economy      2.17          1   5953  
1        Morning           Mumbai  Economy      2.33          1   5953  
2  Early_Morning           Mumbai  Economy      2.17          1   5956  
3      Afternoon           Mumbai  Economy      2.25          1   5955  
4        Morning           Mumbai  Economy      2.33          1   5955

In [6]:
# Clean and normalize column names
df.columns = df.columns.str.strip()      # remove extra spaces
df.columns = df.columns.str.lower()      # convert to lowercase for consistency

print("Cleaned columns:", df.columns.tolist())


Cleaned columns: ['unnamed: 0', 'airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left', 'price']


In [19]:
#  Step 3: Data Cleaning

# Remove extra spaces in column names and make them lowercase
df.columns = df.columns.str.strip().str.lower()

# Define the correct target column for your dataset
TARGET_COLUMN = "price"

# Check that the target variable exists
if TARGET_COLUMN not in df.columns:
    raise ValueError(f" The target column '{TARGET_COLUMN}' is missing from the dataset!")

print(" Columns cleaned and validated successfully.")


 Columns cleaned and validated successfully.


In [20]:
#Define Features and Target

# Drop unnecessary columns
X = df.drop(columns=[TARGET_COLUMN, "unnamed: 0", "flight"])
y = df[TARGET_COLUMN]

print(f" Features and target defined successfully.\nFeature shape: {X.shape}\nTarget shape: {y.shape}")




 Features and target defined successfully.
Feature shape: (300153, 9)
Target shape: (300153,)


In [21]:
#Identify Categorical and Numerical Columns
categorical_features = ["airline", "source_city", "departure_time", "stops",
                        "arrival_time", "destination_city", "class"]
numerical_features = ["duration", "days_left"]

print("Categorical features:", categorical_features)
print("Numerical features:", numerical_features)


Categorical features: ['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']
Numerical features: ['duration', 'days_left']


In [22]:
# Import required modules
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

In [23]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

# Model pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
])

print(" Pipeline created successfully.")

 Pipeline created successfully.


In [26]:
#Split Data and Train Model
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training Data: {X_train.shape}, Testing Data: {X_test.shape}")



Training Data: (240122, 9), Testing Data: (60031, 9)


In [29]:
# Train model
pipeline.fit(X_train, y_train)
print(" Model training complete.")


 Model training complete.


In [30]:
# Make predictions
y_pred = pipeline.predict(X_test)

In [35]:

#  Evaluate the regression model 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(" Model Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.4f}")

#  Save the model

PROJECT_ROOT = os.path.dirname(os.getcwd())

MODEL_PATH = os.path.abspath(
    os.path.join(PROJECT_ROOT, "app", "data", "flight_price_model.pkl")
)

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
joblib.dump(pipeline, MODEL_PATH)

print(f"\nModel saved at: {MODEL_PATH}")

print(f"\n Model trained and saved successfully at: {MODEL_PATH}")




 Model Evaluation Results:
Mean Absolute Error (MAE): 1077.91
Mean Squared Error (MSE): 7771919.08
R² Score: 0.9849

Model saved at: /Users/rohanbk/Documents/DataScienceProjects/predictive-pipeline/app/data/flight_price_model.pkl

 Model trained and saved successfully at: /Users/rohanbk/Documents/DataScienceProjects/predictive-pipeline/app/data/flight_price_model.pkl


In [37]:
import os

model_path = "/Users/rohanbk/Documents/DataScienceProjects/predictive-pipeline/app/data/flight_price_model.pkl"
print("Exists:", os.path.exists(model_path))
print("Size (bytes):", os.path.getsize(model_path) if os.path.exists(model_path) else "Not found")


Exists: True
Size (bytes): 927517930
